In [1]:
import os
from typing import Literal

import polars as pl

import utils

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
num_train: Literal[100] = 100
num_tests: list[int] = [50, 100, 200, 500]
accuracies_home_dir = "accuracies_from_paper"

In [3]:
dfs: list[pl.DataFrame] = []
for num_test in num_tests:
    df = utils.load_all_accuracies(
        os.path.join(accuracies_home_dir, f"m{num_train}"), num_test
    ).with_columns(num_test=pl.lit(num_test))
    df = df.select("num_test").with_columns(df.select(pl.exclude("num_test")))
    dfs.append(df)

In [4]:
accuracy_df = pl.concat(dfs).sort(["num_test", "lm_type", "dataset"])

In [5]:
utils._summarize_differences(
    accuracy_df.with_columns(diff=pl.col("extra") - pl.col("base")),
    groups=("num_test", "lm_type"),
)

num_test,lm_type,mean,se
i32,str,f64,f64
50,"""bert""",0.062064,0.007801
50,"""gpt2""",0.021512,0.00647
100,"""bert""",0.060996,0.007051
100,"""gpt2""",0.024632,0.005623
200,"""bert""",0.040692,0.007439
200,"""gpt2""",0.062892,0.006661
500,"""bert""",0.061296,0.00662
500,"""gpt2""",0.038868,0.005375


In [6]:
utils._summarize_differences(
    accuracy_df.with_columns(diff=pl.col("test") - pl.col("extra")),
    groups=("num_test", "lm_type"),
)

num_test,lm_type,mean,se
i32,str,f64,f64
50,"""bert""",-0.000752,0.006085
50,"""gpt2""",-0.000512,0.002824
100,"""bert""",-0.003712,0.005826
100,"""gpt2""",0.000268,0.002123
200,"""bert""",0.003264,0.006085
200,"""gpt2""",-0.000112,0.001992
500,"""bert""",-0.001592,0.005723
500,"""gpt2""",-0.002076,0.002078
